In [1]:
!pip install pennylane
!pip install numpy
!pip install pandas
!pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)

n_suppliers = 100
n_products = 50
lead_time_mean = 10
lead_time_std = 2
inventory_level_mean = 100
inventory_level_std = 20

data = {
    'supplier_id': np.random.randint(0, n_suppliers, size=n_suppliers * n_products),
    'product_id': np.random.randint(0, n_products, size=n_suppliers * n_products),
    'lead_time': np.random.normal(lead_time_mean, lead_time_std, size=n_suppliers * n_products),
    'inventory_level': np.random.normal(inventory_level_mean, inventory_level_std, size=n_suppliers * n_products),
    'vulnerability': np.random.binomial(1, 0.5, size=n_suppliers * n_products)
}

df = pd.DataFrame(data)
df.to_csv('supply_chain_data.csv', index=False)


In [ ]:
# Problem Statement:
# The goal of this code is to demonstrate the use of Quantum Machine Learning (QML) to analyze vulnerabilities in a supply chain system. The analysis is done by training both quantum and classical machine learning models to predict whether a supplier is vulnerable to disruptions based on features such as lead time, inventory levels, and product identification. A comparison between the performance of the Quantum Support Vector Machine (QSVM) and the Classical Support Vector Machine (SVM) is performed.
# Objective:
# Supply Chain Vulnerability Detection: Use a fictional supply chain dataset to build a QML model to analyze vulnerabilities. The dataset includes information such as supplier ID, product ID, lead time, and inventory level, with a binary label indicating vulnerability.
# Quantum vs. Classical Models: Compare the performance of a QSVM and a classical SVM in predicting supply chain vulnerabilities.
# Dataset Generation: The code also demonstrates how to generate a synthetic supply chain dataset for this problem.
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Load the supply chain dataset
data = pd.read_csv('supply_chain_data.csv')

# Preprocess the data
X = data.drop(['vulnerability'], axis=1)
y = data['vulnerability']

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define the quantum device using PennyLane
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum circuit for embedding the classical data
@qml.qnode(dev)
def quantum_circuit(inputs):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)
        qml.RY(inputs[i], wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Quantum Kernel function
def quantum_kernel(X1, X2):
    result = []
    for x1 in X1:
        row = []
        for x2 in X2:
            row.append(np.dot(quantum_circuit(x1), quantum_circuit(x2)))
        result.append(row)
    return np.array(result)

# Classical Machine Learning (SVM) for comparison
svm = SVC(kernel='linear', C=1)
svm.fit(X_train, y_train)
y_pred_cml = svm.predict(X_test)
accuracy_cml = accuracy_score(y_test, y_pred_cml)
print("Classical Machine Learning Accuracy:", accuracy_cml)

# Quantum Machine Learning (QSVM) using PennyLane
X_train_quantum = quantum_kernel(X_train, X_train)
X_test_quantum = quantum_kernel(X_test, X_train)

qsvm = SVC(kernel='precomputed')
qsvm.fit(X_train_quantum, y_train)
y_pred_qml = qsvm.predict(X_test_quantum)
accuracy_qml = accuracy_score(y_test, y_pred_qml)
print("Quantum Machine Learning Accuracy:", accuracy_qml)

# Compare the accuracy of QML and CML
if accuracy_qml > accuracy_cml:
    print("Quantum Machine Learning outperforms Classical Machine Learning!")
else:
    print("Classical Machine Learning outperforms Quantum Machine Learning!")

# Visualize the results
import matplotlib.pyplot as plt
plt.plot(y_test.reset_index(drop=True), label='True Labels')
plt.plot(y_pred_qml, label='Quantum Machine Learning')
plt.plot(y_pred_cml, label='Classical Machine Learning')
plt.legend()
plt.show()
